## Environment

In [ ]:
!nvidia-smi

Thu Oct  1 01:27:05 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.23.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    40W / 300W |   2943MiB / 16130MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
## colab only
! mkdir -p ~/.kaggle/
! cp kaggle.json ~/.kaggle/
! kaggle competitions download stanford-covid-vaccine  
! unzip train.json.zip
! unzip test.json.zip
! unzip sample_submission.csv.zip

  0% 0.00/132k [00:00<?, ?B/s]
100% 132k/132k [00:00<00:00, 50.6MB/s]
  0% 0.00/89.6k [00:00<?, ?B/s]
100% 89.6k/89.6k [00:00<00:00, 93.5MB/s]
  0% 0.00/132k [00:00<?, ?B/s]
100% 132k/132k [00:00<00:00, 43.0MB/s]
  0% 0.00/132k [00:00<?, ?B/s]
100% 132k/132k [00:00<00:00, 43.2MB/s]
  0% 0.00/89.6k [00:00<?, ?B/s]
100% 89.6k/89.6k [00:00<00:00, 84.5MB/s]
  0% 0.00/89.6k [00:00<?, ?B/s]
100% 89.6k/89.6k [00:00<00:00, 92.0MB/s]
  0% 0.00/132k [00:00<?, ?B/s]
100% 132k/132k [00:00<00:00, 43.1MB/s]
  0% 0.00/89.6k [00:00<?, ?B/s]
100% 89.6k/89.6k [00:00<00:00, 92.2MB/s]
  0% 0.00/132k [00:00<?, ?B/s]
100% 132k/132k [00:00<00:00, 41.4MB/s]
  0% 0.00/89.6k [00:00<?, ?B/s]
100% 89.6k/89.6k [00:00<00:00, 91.0MB/s]
  0% 0.00/132k [00:00<?, ?B/s]
100% 132k/132k [00:00<00:00, 41.4MB/s]
  0% 0.00/132k [00:00<?, ?B/s]
100% 132k/132k [00:00<00:00, 125MB/s]
  0% 0.00/132k [00:00<?, ?B/s]
100% 132k/132k [00:00<00:00, 120MB/s]
  0% 0.00/89.6k [00:00<?, ?B/s]
100% 89.6k/89.6k [00:00<00:00, 92.1MB/s]
  0%

In [ ]:
reset -fs

In [ ]:
## use one-hot input?
one_hot = False

## filter noisy labels?
filter = True

## add additional feature 'pairs'
## if use one_hot, then you cannot add pairs
add_pairs = True

## feature engineering, add 3 features
feature_engineering = True

In [ ]:
import warnings
warnings.filterwarnings('ignore')

#Basic data manipulation libraries
import pandas as pd, numpy as np
import math, json, gc, random, os, sys
from matplotlib import pyplot as plt
from tqdm import tqdm

#Deep Learning Libraries
import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow.keras.backend as K
import tensorflow.keras.layers as L

#Library for model evaluation
from sklearn.model_selection import train_test_split, KFold, RepeatedStratifiedKFold, StratifiedKFold, GroupKFold
from tqdm import tqdm
import glob
from time import time
from scipy.optimize import minimize, fsolve
import datetime

## Data

In [ ]:
train = pd.read_json('train.json', lines=True)
test = pd.read_json('test.json', lines=True)
train = train.query('signal_to_noise >= 1')
# additional features

# def read_bpps_sum(df):
#     bpps_arr = []
#     for mol_id in df.id.to_list():
#         bpps_arr.append(np.load(f"/content/drive/My Drive/bpps/{mol_id}.npy").max(axis=1))
#     return bpps_arr

# def read_bpps_max(df):
#     bpps_arr = []
#     for mol_id in df.id.to_list():
#         bpps_arr.append(np.load(f"/content/drive/My Drive/bpps/{mol_id}.npy").sum(axis=1))
#     return bpps_arr

# def read_bpps_nb(df):
#     # normalized non-zero number
#     # from https://www.kaggle.com/symyksr/openvaccine-deepergcn 
#     bpps_nb_mean = 0.077522 # mean of bpps_nb across all training data
#     bpps_nb_std = 0.08914   # std of bpps_nb across all training data
#     bpps_arr = []
#     for mol_id in df.id.to_list():
#         bpps = np.load(f"/content/drive/My Drive/bpps/{mol_id}.npy")
#         bpps_nb = (bpps > 0).sum(axis=0) / bpps.shape[0]
#         bpps_nb = (bpps_nb - bpps_nb_mean) / bpps_nb_std
#         bpps_arr.append(bpps_nb)
#     return bpps_arr 

# if feature_engineering:
#   train['bpps_sum'] = read_bpps_sum(train)
#   test['bpps_sum'] = read_bpps_sum(test)
#   train['bpps_max'] = read_bpps_max(train)
#   test['bpps_max'] = read_bpps_max(test)
#   train['bpps_nb'] = read_bpps_nb(train)
#   test['bpps_nb'] = read_bpps_nb(test)

sample_sub = pd.read_csv('sample_submission.csv')

target_cols = ['reactivity', 'deg_Mg_pH10', 'deg_pH10', 'deg_Mg_50C', 'deg_50C']

token2int = {x:i for i, x in enumerate('().ACGUBEHIMSX')}

def preprocess_inputs(df, cols=['sequence', 'structure', 'predicted_loop_type']):
    return np.transpose(
        np.array(df[cols].applymap(lambda seq: [token2int[x] for x in seq]).values.tolist()),
        (0, 2, 1))

def fe_preprocess_inputs(df, cols=['sequence', 'structure', 'predicted_loop_type']):
    base_fea = np.transpose(
        np.array(
            df[cols]
            .applymap(lambda seq: [token2int[x] for x in seq])
            .values
            .tolist()
        ),
        (0, 2, 1)
    )
    bpps_sum_fea = np.array(df['bpps_sum'].to_list())[:,:,np.newaxis]
    bpps_max_fea = np.array(df['bpps_max'].to_list())[:,:,np.newaxis]
    bpps_nb_fea = np.array(df['bpps_nb'].to_list())[:,:,np.newaxis]
    return np.concatenate([base_fea,bpps_sum_fea,bpps_max_fea,bpps_nb_fea], 2)

# if feature_engineering:
#   if filter:
#     train_inputs = fe_preprocess_inputs(train[train.signal_to_noise >= 1])
#     train_labels = np.array(train[train.signal_to_noise >= 1][target_cols].values.tolist()).transpose((0, 2, 1))
#     train = train.query('signal_to_noise >= 1')
#   else:
#     train_inputs = preprocess_inputs(train)
#     train_labels = np.array(train[target_cols].values.tolist()).transpose((0, 2, 1))
# else:
#   if filter:
#     train_inputs = preprocess_inputs(train[train.signal_to_noise >= 1])
#     train_labels = np.array(train[train.signal_to_noise >= 1][target_cols].values.tolist()).transpose((0, 2, 1))
#     train = train.query('signal_to_noise >= 1')
#   else:
#     train_inputs = preprocess_inputs(train)
#     train_labels = np.array(train[target_cols].values.tolist()).transpose((0, 2, 1))

In [ ]:
# def to_one_hot(dim3):
#     dim14 = np.zeros(14)
#     for num in dim3:
#         dim14[num] = 1
#     return dim14
# if one_hot:
#     train_inputs = np.apply_along_axis(to_one_hot, 2, train_inputs)

In [ ]:
# def compute_base_pairings(arr, public=True):
#     if public:
#         pairings = np.zeros((len(arr), 107), dtype=np.int8)
#     else:
#         pairings = np.zeros((len(arr), 130), dtype=np.int8)
    
#     for i in range(len(arr)):
#         temp_struc = arr[i, :, 1]
#         temp_nucle = arr[i, :, 0]
#         temp = np.zeros(len(temp_struc), dtype=np.int8)
#         temp_pair = np.zeros(len(temp_struc), dtype=np.int8)
        
#         p = []
#         count = 0
        
#         # map pairings to same integer
#         for j in range(len(temp_struc)):
#             if temp_struc[j] == 0:
#                 count += 1
#                 p.append(count)
#                 temp[j] = count
#             elif temp_struc[j] == 1:
#                 temp[j] = p.pop()
        
#         # map pair nucleotide
#         for j in range(1, count+1):
#             pair_idx = np.where(temp==j)[0]
            
#             if len(pair_idx) > 1:
#                 temp_pair[pair_idx[0]] = temp_nucle[pair_idx[1]]
#                 temp_pair[pair_idx[1]] = temp_nucle[pair_idx[0]]
#             else:
#                 temp_pair[pair_idx[0]] = 3
    
#         pairings[i] = temp_pair
    
#     return pairings

In [ ]:
# #get different test sets and process each
public_df = test.query("seq_length == 107").copy()
private_df = test.query("seq_length == 130").copy()
# if feature_engineering:
#   public_inputs = fe_preprocess_inputs(public_df)
#   private_inputs = fe_preprocess_inputs(private_df)
# else:
#   public_inputs = preprocess_inputs(public_df)
#   private_inputs = preprocess_inputs(private_df)

# if one_hot:
#   public_inputs = np.apply_along_axis(to_one_hot, 2, public_inputs)
#   private_inputs = np.apply_along_axis(to_one_hot, 2, private_inputs)

In [ ]:
# if add_pairs:
#   train_pairs = compute_base_pairings(train_inputs)
#   public_test_pairs = compute_base_pairings(public_inputs)
#   private_test_pairs = compute_base_pairings(private_inputs, public=False)
#   train_inputs = np.concatenate((train_inputs, np.reshape(train_pairs, (train_pairs.shape[0], train_pairs.shape[1], 1))), axis=2)
#   public_inputs = np.concatenate((public_inputs, np.reshape(public_test_pairs, (public_test_pairs.shape[0], public_test_pairs.shape[1], 1))), axis=2)
#   private_inputs = np.concatenate((private_inputs, np.reshape(private_test_pairs, (private_test_pairs.shape[0], private_test_pairs.shape[1], 1))), axis=2)

In [ ]:
train_inputs = np.load('/content/drive/My Drive/open_vaccine_data_added_fe/train_inputs.npy')
train_labels = np.load('/content/drive/My Drive/open_vaccine_data_added_fe/train_labels.npy')
private_inputs = np.load('/content/drive/My Drive/open_vaccine_data_added_fe/private_inputs.npy')
public_inputs = np.load('/content/drive/My Drive/open_vaccine_data_added_fe/public_inputs.npy')

In [ ]:
from sklearn.cluster import KMeans
kmeans_model = KMeans(n_clusters=200, random_state=42).fit(train_inputs[:,:,0])
train['cluster_id'] = kmeans_model.labels_

## Loss

In [ ]:
def rmse(y_actual, y_pred):
    mse = tf.keras.losses.mean_squared_error(y_actual, y_pred)
    return K.sqrt(mse)
    
def MCRMSE(y_actual, y_pred, num_scored=len(target_cols)):
    score = 0
    for i in [range(num_scored)]:
        score += rmse(y_actual[:, :, i], y_pred[:, :, i]) / num_scored
    return score

def MCRMSE_3(y_actual, y_pred):
    score = 0
    for i in [0,1,2,3,4]:
        score += rmse(y_actual[:, :, i], y_pred[:, :, i]) / 5
    return score

## Models

In [ ]:
def gru_layer(hidden_dim, dropout):
    return tf.keras.layers.Bidirectional(
                                tf.keras.layers.GRU(hidden_dim,
                                dropout=dropout,
                                return_sequences=True,
                                kernel_initializer = 'orthogonal'))

def lstm_layer(hidden_dim, dropout):
    return tf.keras.layers.Bidirectional(
                                tf.keras.layers.LSTM(hidden_dim,
                                dropout=dropout,
                                return_sequences=True,
                                kernel_initializer = 'orthogonal'))

def build_model(gru=1,seq_len=107, pred_len=68, dropout=0.4,
                embed_dim=100, hidden_dim=128):
    if one_hot:
      inputs = tf.keras.layers.Input(shape=(seq_len, 14))
    elif feature_engineering:
      inputs = tf.keras.layers.Input(shape=(seq_len, 7))
    elif add_pairs:
      inputs = tf.keras.layers.Input(shape=(seq_len, 4))
    else:
      inputs = tf.keras.layers.Input(shape=(seq_len, 3))

    embed = tf.keras.layers.Embedding(input_dim=len(token2int), output_dim=embed_dim)(inputs)
    reshaped = tf.reshape(
        embed, shape=(-1, embed.shape[1],  embed.shape[2] * embed.shape[3]))
    
    #reshaped = tf.keras.layers.SpatialDropout1D(.2)(reshaped)
    
    if gru==0:
        hidden = gru_layer(hidden_dim, dropout)(reshaped)
        hidden = gru_layer(hidden_dim, dropout)(hidden)
        hidden = gru_layer(hidden_dim, dropout)(hidden)
        
    elif gru==1:
        hidden = lstm_layer(hidden_dim, dropout)(reshaped)
        hidden = lstm_layer(hidden_dim, dropout)(hidden)
        hidden = lstm_layer(hidden_dim, dropout)(hidden)
        
    elif gru==2:
        hidden = gru_layer(hidden_dim, dropout)(reshaped)
        hidden = lstm_layer(hidden_dim, dropout)(hidden)
        hidden = lstm_layer(hidden_dim, dropout)(hidden)
        
    elif gru==3:
        hidden = lstm_layer(hidden_dim, dropout)(reshaped)
        hidden = gru_layer(hidden_dim, dropout)(hidden)
        hidden = gru_layer(hidden_dim, dropout)(hidden)
        
    elif gru==4:
        hidden = lstm_layer(hidden_dim, dropout)(reshaped)
        hidden = gru_layer(hidden_dim, dropout)(hidden)
        hidden = lstm_layer(hidden_dim, dropout)(reshaped)
    
    #only making predictions on the first part of each sequence
    truncated = hidden[:, :pred_len]
    
    out1 = tf.keras.layers.Dense(1024, activation='relu')(truncated)
    out2 = tf.keras.layers.Dropout(0.4)(out1)
    out = tf.keras.layers.Dense(5, activation='linear')(out2)

    model = tf.keras.Model(inputs=inputs, outputs=out)

    #some optimizers
    adam = tf.optimizers.Adam()
    radam = tfa.optimizers.RectifiedAdam()
    lookahead = tfa.optimizers.Lookahead(adam, sync_period=6)
    ranger = tfa.optimizers.Lookahead(radam, sync_period=6)
    
    model.compile(optimizer = adam, loss=MCRMSE_3)
    
    return model

In [ ]:
def build_wavenet_model(seq_len = 107, pred_len = 68, embed_dim = 100, dropout = 0.1):
    
    def wave_block(x, filters, kernel_size, n):
        dilation_rates = [2 ** i for i in range(n)]
        x = tf.keras.layers.Conv1D(filters = filters, 
                                   kernel_size = 1,
                                   padding = 'same')(x)
        res_x = x
        for dilation_rate in dilation_rates:
            tanh_out = tf.keras.layers.Conv1D(filters = filters,
                              kernel_size = kernel_size,
                              padding = 'same', 
                              activation = 'tanh', 
                              dilation_rate = dilation_rate)(x)
            sigm_out = tf.keras.layers.Conv1D(filters = filters,
                              kernel_size = kernel_size,
                              padding = 'same',
                              activation = 'sigmoid', 
                              dilation_rate = dilation_rate)(x)
            x = tf.keras.layers.Multiply()([tanh_out, sigm_out])
            x = tf.keras.layers.Conv1D(filters = filters,
                       kernel_size = 1,
                       padding = 'same')(x)
            res_x = tf.keras.layers.Add()([res_x, x])
        return res_x
    if one_hot:
      inputs = tf.keras.layers.Input(shape = (seq_len, 14))
    elif feature_engineering:
      inputs = tf.keras.layers.Input(shape = (seq_len, 7))
    elif add_pairs:
      inputs = tf.keras.layers.Input(shape = (seq_len, 4))
    else:
      inputs = tf.keras.layers.Input(shape = (seq_len, 3))

    embed = tf.keras.layers.Embedding(input_dim = len(token2int), output_dim = embed_dim)(inputs)
    reshaped = tf.reshape(embed, shape = (-1, embed.shape[1], embed.shape[2] * embed.shape[3]))
    reshaped = tf.keras.layers.SpatialDropout1D(dropout)(reshaped)
    
    x = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(256, 
                                                          dropout = dropout, 
                                                          return_sequences = True, 
                                                          kernel_initializer = 'orthogonal'))(reshaped)
    x = wave_block(x, 16, 3, 12)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(dropout)(x)
    x = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(256, 
                                                          dropout = dropout, 
                                                          return_sequences = True, 
                                                          kernel_initializer = 'orthogonal'))(x)
    x = wave_block(x, 32, 3, 8)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(dropout)(x)
    x = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(256, 
                                                          dropout = dropout, 
                                                          return_sequences = True, 
                                                          kernel_initializer = 'orthogonal'))(x)
    x = wave_block(x, 64, 3, 4)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(dropout)(x)
    x = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(256, 
                                                          dropout = dropout, 
                                                          return_sequences = True, 
                                                          kernel_initializer = 'orthogonal'))(x)

    truncated = x[:, :pred_len]
    out1 = tf.keras.layers.Dense(1024, activation='relu')(truncated)
    out2 = tf.keras.layers.Dropout(0.4)(out1)
    out = tf.keras.layers.Dense(5, activation = 'linear')(out2)
    model = tf.keras.models.Model(inputs = inputs, outputs = out)
    opt = tf.keras.optimizers.Adam(learning_rate = 0.001)
    opt = tfa.optimizers.SWA(opt)
    model.compile(optimizer = opt,
                  loss = MCRMSE_3)
    
    return model


## Utilities

In [ ]:
from keras.callbacks import Callback
from keras import backend as K
class CosineAnnealingScheduler(Callback):
    """Cosine annealing scheduler.
    """

    def __init__(self, T_max, eta_max, eta_min=0, verbose=0):
        super(CosineAnnealingScheduler, self).__init__()
        self.T_max = T_max
        self.eta_max = eta_max
        self.eta_min = eta_min
        self.verbose = verbose

    def on_epoch_begin(self, epoch, logs=None):
        if not hasattr(self.model.optimizer, 'lr'):
            raise ValueError('Optimizer must have a "lr" attribute.')
        lr = self.eta_min + (self.eta_max - self.eta_min) * (1 + math.cos(math.pi * epoch / self.T_max)) / 2
        K.set_value(self.model.optimizer.lr, lr)
        if self.verbose > 0:
            print('\nEpoch %05d: CosineAnnealingScheduler setting learning '
                  'rate to %s.' % (epoch + 1, lr))

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        logs['lr'] = K.get_value(self.model.optimizer.lr)

In [ ]:
## choices are: 0, 1, 2, 3, 4, 'wn'
from sklearn.cluster import KMeans

def training(model_type, Verbose, seed=101):
  kmeans_model = KMeans(n_clusters=200, random_state=seed).fit(train_inputs[:,:,0])
  train['cluster_id'] = kmeans_model.labels_
  model_private_preds = np.zeros((private_df.shape[0], 130, 5))
  model_public_preds = np.zeros((public_df.shape[0], 107, 5))
  oof = []
  labels = []
  if model_type == 'wn':
    name = 'wavenet'
  elif model_type == 0:
    name = 'gru'
  elif model_type == 1:
    name = 'lstm'
  elif model_type == 2:
    name = 'hybrid1'
  elif model_type == 3:
    name = 'hybrid2'
  elif model_type == 4:
    name = 'hybrid3'
  print(f'Training 5-fold {name} now......')
  histories = []
  rskf = RepeatedStratifiedKFold(5, n_repeats = 1, random_state = 42)
  gkf = GroupKFold(n_splits=5)
  kf = rskf.split(train_inputs, train['SN_filter'])
  for fold, (train_index, val_index) in enumerate(gkf.split(train_inputs, train['reactivity'], train['cluster_id'])):
      train_ = train_inputs[train_index]
      train_labs = train_labels[train_index]
      val_ = train_inputs[val_index]
      val_labs = train_labels[val_index]

      # lr_callback = CosineAnnealingScheduler(T_max=100, eta_max=1e-3, eta_min=1e-8)

      if model_type == 'wn':
        model = build_wavenet_model()
      else:
        model = build_model(gru=model_type)

      sv = tf.keras.callbacks.ModelCheckpoint(f'model_{name}_{fold}.h5')

      history = model.fit(
          train_, train_labs, 
          validation_data=(val_,val_labs),
          batch_size=128,
          epochs=200,
          #callbacks=[lr_callback,sv],
          callbacks=[tf.keras.callbacks.ReduceLROnPlateau(patience=10),sv],
          verbose = Verbose
      )
      histories.append(history)
      fold_loss = min(history.history['val_loss'])
      print(f'Best Validation Loss for fold {fold} is {fold_loss}')

      ## Calculate OOF
      model.load_weights(f'model_{name}_{fold}.h5') # load the model
      oof.append(model.predict(val_)) # append to oof
      labels.append(val_labs)

      if model_type == 'wn':
        model_short = build_wavenet_model(seq_len=107, pred_len=107)
        model_long = build_wavenet_model(seq_len=130, pred_len=130)
      else:
        model_short = build_model(gru=model_type, seq_len=107, pred_len=107)
        model_long = build_model(gru=model_type, seq_len=130, pred_len=130)

      model_short.load_weights(f'model_{name}_{fold}.h5')
      model_public_pred = model_short.predict(public_inputs) / 5
      
      model_long.load_weights(f'model_{name}_{fold}.h5')
      model_private_pred = model_long.predict(private_inputs) / 5 

      model_public_preds += model_public_pred
      model_private_preds += model_private_pred
  
  print(f"Overall mean {name} MCRMSE: {np.mean([min(history.history['val_loss']) for history in histories])}")
  return oof, labels, model_public_preds, model_private_preds

## Wavenet Training

In [ ]:
oof_wn, labels_wn, wn_public_preds, wn_private_preds = training(model_type='wn', Verbose=0)

Training 5-fold wavenet now......
Best Validation Loss for fold 0 is 0.2346225529909134
Best Validation Loss for fold 1 is 0.2530500888824463
Best Validation Loss for fold 2 is 0.24116666615009308
Best Validation Loss for fold 3 is 0.248418390750885
Best Validation Loss for fold 4 is 0.21891511976718903
Overall wavenet MCRMSE: 0.23923456370830537


In [ ]:
# preds_wn_df_seed1 = pd.concat(arrange_pred(wn_public_preds, wn_private_preds))
# submission_seed1 = sample_sub[['id_seqpos']].merge(preds_wn_df, on=['id_seqpos'])

In [ ]:
preds_wn_df_seed2 = pd.concat(arrange_pred(wn_public_preds, wn_private_preds))
submission_seed2 = sample_sub[['id_seqpos']].merge(preds_wn_df, on=['id_seqpos'])

In [ ]:
submission = submission_seed1.copy()

In [ ]:
submission.iloc[:, 1:] = submission_seed1.iloc[:, 1:] * .5 + submission_seed2.iloc[:, 1:] * .5

In [ ]:
submission.to_csv('wavenet_2seeds_cv237.csv', index=False)

* 0.3386 Wavenet, patience = 7
* 0.3450 Wavenet, patience = 7, dropout=0.2
* 0.21360034, patience=7, original
* 0.209356, patience=7, added another dense
* 0.2085, patience=7, dense = 1024
* 0.2079, patience=7, dense = 1024
* 0.2075, patience=10, dense = 1024
* 0.2074, patience=10, feature engineering

## GRU Training

In [ ]:
def MCRMSE_3_(y_actual, y_pred):
    score = 0
    for i in [0,1,3]:
        score += rmse(y_actual[:, :, i], y_pred[:, :, i]) / 3
    return np.mean(score)

In [ ]:
def arrange_pred(public_preds, private_preds):
  preds_model = []
  for df, preds in [(public_df, public_preds), (private_df, private_preds)]:
      for i, uid in enumerate(df.id):
          single_pred = preds[i]
          single_df = pd.DataFrame(single_pred, columns=target_cols)
          single_df['id_seqpos'] = [f'{uid}_{x}' for x in range(single_df.shape[0])]
          preds_model.append(single_df)
  return preds_model

In [ ]:
final_sub = sample_sub.copy()

In [ ]:
for i in [1,2,3,4,5]:
  print(f'seed is {i}')
  oof_gru, labels_gru, gru_public_preds, gru_private_preds = training(model_type=2, Verbose=0, seed=i)
  oof_gru_overall = np.concatenate((oof_gru[0], oof_gru[1], oof_gru[2], oof_gru[3], oof_gru[4]))
  ground_truth_label = np.concatenate((labels_gru[0], labels_gru[1], labels_gru[2], labels_gru[3], labels_gru[4]))
  print(f'OVERALL loss is {MCRMSE_3_(ground_truth_label, oof_gru_overall)}')
  preds_gru_df = pd.concat(arrange_pred(gru_public_preds, gru_private_preds))
  submission = sample_sub[['id_seqpos']].merge(preds_gru_df, on=['id_seqpos'])
  final_sub.iloc[:, 1:] += .2 * submission.iloc[:, 1:]

seed is 1
Training 5-fold hybrid1 now......
Best Validation Loss for fold 0 is 0.25155872106552124
Best Validation Loss for fold 1 is 0.22571605443954468
Best Validation Loss for fold 2 is 0.25860220193862915
Best Validation Loss for fold 3 is 0.2503390312194824
Best Validation Loss for fold 4 is 0.2517712116241455
Overall mean hybrid1 MCRMSE: 0.2475974440574646
OVERALL loss is 0.24315397441387177
seed is 2
Training 5-fold hybrid1 now......
Best Validation Loss for fold 0 is 0.2515604496002197
Best Validation Loss for fold 1 is 0.26008865237236023
Best Validation Loss for fold 2 is 0.23867502808570862
Best Validation Loss for fold 3 is 0.24746820330619812
Best Validation Loss for fold 4 is 0.28446507453918457
Overall mean hybrid1 MCRMSE: 0.2564514815807343
OVERALL loss is 0.25104594230651855
seed is 3
Training 5-fold hybrid1 now......
Best Validation Loss for fold 0 is 0.25204551219940186
Best Validation Loss for fold 1 is 0.23795200884342194
Best Validation Loss for fold 2 is 0.242287

In [ ]:
final_sub.head()

,id_seqpos,reactivity,deg_Mg_pH10,deg_pH10,deg_Mg_50C,deg_50C
0,id_00073f8be_0,0.637045,0.656401,2.042983,0.533870,0.748103
1,id_00073f8be_1,1.983767,2.809911,3.978055,2.927414,2.659771
2,id_00073f8be_2,1.360241,0.603129,0.723768,0.721851,0.794325
3,id_00073f8be_3,1.278732,1.068196,1.157663,1.536598,1.550203
4,id_00073f8be_4,0.857197,0.599222,0.586575,0.853780,0.826137


In [ ]:
final_sub.to_csv('5fold_5seed_cv245_lstm_only.csv', index=False)

In [ ]:
# submit
!kaggle competitions submit -c stanford-covid-vaccine -f 5fold_5seed_cv245_lstm_only.csv -m "cv245"

100% 49.6M/49.6M [00:05<00:00, 9.59MB/s]
Successfully submitted to OpenVaccine: COVID-19 mRNA Vaccine Degradation Prediction

* 3415 patient=7
* 2167 filtered
* 2142 filtered with another dense512 + dropout0.4(best)
* 2131 filtered with another dense1024 + dropout0.4(best)
* 2103 filtered with another dense1024 + dropout0.4(best), patient=12
* 2104 feature engineering

## LSTM Training

In [ ]:
oof_lstm, labels_lstm, lstm_public_preds, lstm_private_preds = training(model_type=1, Verbose=0)

Training 5-fold lstm now......
Best Validation Loss for fold 0 is 0.25026124715805054
Best Validation Loss for fold 1 is 0.2479071319103241
Best Validation Loss for fold 2 is 0.2454013079404831
Best Validation Loss for fold 3 is 0.24307531118392944
Best Validation Loss for fold 4 is 0.2598634660243988
Overall lstm MCRMSE: 0.2493016928434372


* 0.3925 ReduceLrOnPlateu
* 0.218 dense 512
* 0.216 dense 1024
* 0.2155 feature engineering

## Hybrid Training 1

In [ ]:
oof_hyd1, labels_hyd1, hyd1_public_preds, hyd1_private_preds = training(model_type=2, Verbose=0)

Training 5-fold hybrid1 now......
Best Validation Loss for fold 0 is 0.24281279742717743
Best Validation Loss for fold 1 is 0.24454864859580994
Best Validation Loss for fold 2 is 0.2439412772655487
Best Validation Loss for fold 3 is 0.2362826019525528
Best Validation Loss for fold 4 is 0.2576320171356201
Overall hybrid1 MCRMSE: 0.2450434684753418


0.21655

0.2141

## Hybrid Training 2

In [ ]:
oof_hyd2, labels_hyd2, hyd2_public_preds, hyd2_private_preds = training(model_type=3, Verbose=0)

Training 5-fold hybrid2 now......
Best Validation Loss for fold 0 is 0.24593289196491241
Best Validation Loss for fold 1 is 0.23983217775821686
Best Validation Loss for fold 2 is 0.2403288036584854
Best Validation Loss for fold 3 is 0.23508141934871674
Best Validation Loss for fold 4 is 0.25233274698257446
Overall hybrid2 MCRMSE: 0.24270160794258117


* 0.213325 512
* 0.21236 1024
* 0.210 patience = 12 

## Hybrid Training 3

In [ ]:
oof_hyd3, labels_hyd3, hyd3_public_preds, hyd3_private_preds = training(model_type=4, Verbose=0)

Training 5-fold hybrid3 now......
Best Validation Loss for fold 0 is 0.2532396912574768
Best Validation Loss for fold 1 is 0.24669629335403442
Best Validation Loss for fold 2 is 0.24496805667877197
Best Validation Loss for fold 3 is 0.24376976490020752
Best Validation Loss for fold 4 is 0.26051726937294006
Overall hybrid3 MCRMSE: 0.24983821511268617


* 0.220336
* 0.21945
* 0.2193

## OOF Evaluation

In [ ]:
if filter:
  oof_wn_overall = np.concatenate((oof_wn[0], oof_wn[1], oof_wn[2], oof_wn[3], oof_wn[4]))
  oof_gru_overall = np.concatenate((oof_gru[0], oof_gru[1], oof_gru[2], oof_gru[3], oof_gru[4]))
  oof_lstm_overall = np.concatenate((oof_lstm[0], oof_lstm[1], oof_lstm[2], oof_lstm[3], oof_lstm[4]))
  oof_hyd1_overall = np.concatenate((oof_hyd1[0], oof_hyd1[1], oof_hyd1[2], oof_hyd1[3], oof_hyd1[4]))
  oof_hyd2_overall = np.concatenate((oof_hyd2[0], oof_hyd2[1], oof_hyd2[2], oof_hyd2[3], oof_hyd2[4]))
  oof_hyd3_overall = np.concatenate((oof_hyd3[0], oof_hyd3[1], oof_hyd3[2], oof_hyd3[3], oof_hyd3[4]))
  ground_truth_label = np.concatenate((labels_wn[0], labels_wn[1], labels_wn[2], labels_wn[3], labels_wn[4]))
else:
  oof_wn_overall = np.array(oof_wn).reshape(-1, 68, 5)
  oof_gru_overall = np.array(oof_gru).reshape(-1, 68, 5)
  oof_lstm_overall = np.array(oof_lstm).reshape(-1, 68, 5)
  oof_hyd1_overall = np.array(oof_hyd1).reshape(-1, 68, 5)
  oof_hyd2_overall = np.array(oof_hyd2).reshape(-1, 68, 5)
  oof_hyd3_overall = np.array(oof_hyd3).reshape(-1, 68, 5)
  ground_truth_label = np.array(labels_wn).reshape(-1, 68, 5)

In [ ]:
def MCRMSE_(y_actual, y_pred):
    score = 0
    for i in range(5):
        score += rmse(y_actual[:, :, i], y_pred[:, :, i]) / 5
    return np.mean(score)

def MCRMSE_3(y_actual, y_pred):
    score = 0
    for i in [0,1,3]:
        score += rmse(y_actual[:, :, i], y_pred[:, :, i]) / 3
    return np.mean(score)

In [ ]:
for index, i in enumerate([oof_wn_overall, oof_gru_overall, oof_lstm_overall, oof_hyd1_overall, oof_hyd2_overall, oof_hyd3_overall]):
  print(f'{index+1}. ', MCRMSE_3(ground_truth_label, i))

1.  0.24083109
2.  0.24245878
3.  0.25006413
4.  0.2460459
5.  0.24346918
6.  0.25051346


In [ ]:
## output oof

np.save('/content/drive/My Drive/open_vaccine_data_added_fe/oof_wn_fe.npy', oof_wn_overall)
np.save('/content/drive/My Drive/open_vaccine_data_added_fe/oof_gru_fe.npy', oof_gru_overall)
np.save('/content/drive/My Drive/open_vaccine_data_added_fe/oof_lstm_fe.npy', oof_lstm_overall)
np.save('/content/drive/My Drive/open_vaccine_data_added_fe/oof_hyd1_fe.npy', oof_hyd1_overall)
np.save('/content/drive/My Drive/open_vaccine_data_added_fe/oof_hyd2_fe.npy', oof_hyd2_overall)
np.save('/content/drive/My Drive/open_vaccine_data_added_fe/oof_hyd3_fe.npy', oof_hyd3_overall)

In [ ]:
# np.save('ground_truth_label.npy', ground_truth_label)

In [ ]:
def func_numpy_metric(weights):
    w1, w2, w3, w4, w5, w6 = weights
    oof_blend = w1 * oof_wn_overall + w2 * oof_gru_overall + w3 * oof_lstm_overall + w4 * oof_hyd1_overall + w5 * oof_hyd2_overall + w6 * oof_hyd3_overall
    score = MCRMSE_3(ground_truth_label, oof_blend)
    return score

In [ ]:
print('\n Finding Blending Weights ...')
from tqdm import tqdm
res_list = []
weights_list = []

for k in tqdm(range(1000)):
    a = np.random.randint(0, 100, 6)
    starting_values = a / np.sum(a)
    bounds = [(0, 1)] * 6
    
    res = minimize(func_numpy_metric,
                   starting_values,
                   method='L-BFGS-B',
                   bounds=bounds,
                   options={'disp': False,
                            'maxiter': 100000}) 
    
    res_list.append(res['fun'])
    weights_list.append(res['x'])
    
bestSC   = np.min(res_list)
bestWght = weights_list[np.argmin(res_list)]
weights  = bestWght
blend_score = round(bestSC, 6)

  0%|          | 1/1000 [00:00<02:19,  7.18it/s]


 Finding Blending Weights ...


100%|██████████| 1000/1000 [08:20<00:00,  2.00it/s]


In [ ]:
print('\n Ensemble Score: {best_score}'.format(best_score=bestSC))
print('\n Best Weights: {weights}'.format(weights=bestWght))


 Ensemble Score: 0.23115622997283936

 Best Weights: [0.309375   0.25625    0.084375   0.153125   0.109375   0.08306165]


In [ ]:
sum(weights)

0.9955616549199191

## Submission

In [ ]:
def arrange_pred(public_preds, private_preds):
  preds_model = []
  for df, preds in [(public_df, public_preds), (private_df, private_preds)]:
      for i, uid in enumerate(df.id):
          single_pred = preds[i]
          single_df = pd.DataFrame(single_pred, columns=target_cols)
          single_df['id_seqpos'] = [f'{uid}_{x}' for x in range(single_df.shape[0])]
          preds_model.append(single_df)
  return preds_model

In [ ]:
preds_wn_df = pd.concat(arrange_pred(wn_public_preds, wn_private_preds))
preds_gru_df = pd.concat(arrange_pred(gru_public_preds, gru_private_preds))
preds_lstm_df = pd.concat(arrange_pred(lstm_public_preds, lstm_private_preds))
preds_hyd1_df = pd.concat(arrange_pred(hyd1_public_preds, hyd1_private_preds))
preds_hyd2_df = pd.concat(arrange_pred(hyd2_public_preds, hyd2_private_preds))
preds_hyd3_df = pd.concat(arrange_pred(hyd3_public_preds, hyd3_private_preds))

In [ ]:
weights = weights / np.sum(weights) # making sure it's 1

In [ ]:
blend_preds_df = preds_hyd3_df.copy()
blend_preds_df.iloc[:, :-1] = 0
for index, i in enumerate([preds_wn_df, preds_gru_df, preds_lstm_df, preds_hyd1_df, preds_hyd2_df, preds_hyd3_df]):
  blend_preds_df.iloc[:, :-1] += i.iloc[:,:-1] * weights[index]

In [ ]:
blend_preds_df.head()

,reactivity,deg_Mg_pH10,deg_pH10,deg_Mg_50C,deg_50C,id_seqpos
0,0.688670,0.652019,-0.020325,0.548482,0.042420,id_00073f8be_0
1,2.138515,2.948593,0.023414,3.085091,0.040201,id_00073f8be_1
2,1.444244,0.622930,0.036847,0.744493,0.039590,id_00073f8be_2
3,1.257418,1.064093,-0.008306,1.532421,0.075368,id_00073f8be_3
4,0.854573,0.596857,-0.006440,0.851687,0.026148,id_00073f8be_4


In [ ]:
submission = sample_sub[['id_seqpos']].merge(blend_preds_df, on=['id_seqpos'])

In [ ]:
submission.to_csv('/content/drive/My Drive/open_vaccine_data_added_fe/sub_fe_new_cv.csv', index=False)

In [ ]:
submission.to_csv('sub_fe.csv', index=False)

In [ ]:
# submit
!kaggle competitions submit -c stanford-covid-vaccine -f wavenet_2seeds_cv237.csv -m "cv237"

100% 50.8M/50.8M [00:04<00:00, 11.6MB/s]
Successfully submitted to OpenVaccine: COVID-19 mRNA Vaccine Degradation Prediction

In [ ]:
submission

,reactivity,deg_Mg_pH10,deg_pH10,deg_Mg_50C,deg_50C
0,0.763814,0.692169,0.069476,0.576933,-0.094168
1,2.381897,3.314302,-0.035656,3.318505,-0.389209
2,1.534115,0.624643,-0.118847,0.732306,-0.033215
3,1.322329,1.162570,-0.043088,1.683879,-0.111748
4,0.844963,0.592926,-0.008817,0.852003,-0.097187
...,...,...,...,...,...
457948,0.496888,0.667506,0.084097,0.965359,-0.050289
457949,0.706034,1.056895,0.118600,1.342684,-0.011633
457950,0.829533,0.390117,0.094393,0.733908,-0.008469
457951,0.467425,0.433830,0.099879,0.715381,-0.021365
